# Student Database (MongoDB)

In [7]:
import pymongo

In [8]:
client = pymongo.MongoClient("mongodb://localhost:27017/")

In [9]:
db = client.get_database('Project')

In [10]:
records = db.Students

In [19]:
#to check connection
x = records.find_one(32)
print(x)

{'_id': 32, 'name': 'Gwen Honig', 'scores': [{'score': 87.14345376886205, 'type': 'exam'}, {'score': 99.45824441135635, 'type': 'quiz'}, {'score': 76.66460454219344, 'type': 'homework'}]}


## 1)      Find the student name who scored maximum scores in all (exam, quiz and homework)?

In [25]:
#$unwind operator is used to deconstruct the documents in MongoDB.
#If the operand resolves to a non-empty array, $first returns the first element in the array.
#a group specification must include an _id, so "_id" should be added.

for i in records.aggregate([{"$unwind":"$scores"},{"$group":{"_id":"$_id", "name":{"$first":"$name"}, "max":{"$sum":"$scores.score"},}},{"$sort":{"max":-1}},{"$limit":1}]):
    print(i)

{'_id': 13, 'name': 'Jessika Dagenais', 'max': 275.94947751518237}


## 2)      Find students who scored below average in the exam and pass mark is 40%?

In [26]:
#$match operator filters the documents to pass only those documents that match the specified condition (s) to the next pipeline stage.
for x in records.aggregate([{"$unwind":"$scores"},{"$match":{"scores.type":"exam", "scores.score":{"$gt":40, "$lt":60}}}]):
    print(x)

{'_id': 5, 'name': 'Wilburn Spiess', 'scores': {'score': 44.87186330181261, 'type': 'exam'}}
{'_id': 10, 'name': 'Denisha Cast', 'scores': {'score': 45.61876862259409, 'type': 'exam'}}
{'_id': 12, 'name': 'Quincy Danaher', 'scores': {'score': 54.29841278520669, 'type': 'exam'}}
{'_id': 19, 'name': 'Gisela Levin', 'scores': {'score': 44.51211101958831, 'type': 'exam'}}
{'_id': 20, 'name': 'Tressa Schwing', 'scores': {'score': 42.17439799514388, 'type': 'exam'}}
{'_id': 21, 'name': 'Rosana Vales', 'scores': {'score': 46.2289476258328, 'type': 'exam'}}
{'_id': 23, 'name': 'Tamika Schildgen', 'scores': {'score': 45.65432764125526, 'type': 'exam'}}
{'_id': 29, 'name': 'Gwyneth Garling', 'scores': {'score': 48.36644963899371, 'type': 'exam'}}
{'_id': 42, 'name': 'Kayce Kenyon', 'scores': {'score': 44.62441703708117, 'type': 'exam'}}
{'_id': 45, 'name': 'Terica Brugger', 'scores': {'score': 42.1011312120801, 'type': 'exam'}}
{'_id': 57, 'name': 'Chad Rahe', 'scores': {'score': 40.845720273667

## 3)      Find students who scored below pass mark and assigned them as fail, and above pass mark as pass in all the categories.

In [30]:
for y in records.aggregate([{"$set":{"scores":{"$arrayToObject":[{"$map": 
           {"input": "$scores",
            "as": "s",
            "in": {"k": "$$s.type", "v": "$$s.score"}}}]}}},{"$project":
  {
     "_id":1,
     "name":1,
     "result":{"$cond":{"if": {"$and" : [{"$gte": ["$scores.exam", 40]}, {"$gte": ["$scores.quiz", 40]}, {"$gte": [ "$scores.homework", 40]}]},
                    "then" :"pass",
                    "else":"fail"}}}}]):
    print(y)

{'_id': 0, 'name': 'aimee Zank', 'result': 'fail'}
{'_id': 1, 'name': 'Aurelia Menendez', 'result': 'pass'}
{'_id': 2, 'name': 'Corliss Zuk', 'result': 'fail'}
{'_id': 3, 'name': 'Bao Ziglar', 'result': 'fail'}
{'_id': 4, 'name': 'Zachary Langlais', 'result': 'fail'}
{'_id': 5, 'name': 'Wilburn Spiess', 'result': 'fail'}
{'_id': 6, 'name': 'Jenette Flanders', 'result': 'fail'}
{'_id': 7, 'name': 'Salena Olmos', 'result': 'pass'}
{'_id': 8, 'name': 'Daphne Zheng', 'result': 'fail'}
{'_id': 9, 'name': 'Sanda Ryba', 'result': 'fail'}
{'_id': 10, 'name': 'Denisha Cast', 'result': 'pass'}
{'_id': 11, 'name': 'Marcus Blohm', 'result': 'pass'}
{'_id': 12, 'name': 'Quincy Danaher', 'result': 'pass'}
{'_id': 13, 'name': 'Jessika Dagenais', 'result': 'pass'}
{'_id': 14, 'name': 'Alix Sherrill', 'result': 'fail'}
{'_id': 15, 'name': 'Tambra Mercure', 'result': 'fail'}
{'_id': 16, 'name': 'Dodie Staller', 'result': 'fail'}
{'_id': 17, 'name': 'Fletcher Mcconnell', 'result': 'fail'}
{'_id': 18, 'na

## 4)       Find the total and average of the exam, quiz and homework and store them in a separate collection

In [32]:
average = db.student_avg_total

In [35]:
data = []
for j in records.aggregate([{"$unwind":"$scores"},{"$group":{"_id":"$_id","name":{"$first":"$name"},"total":{"$sum":"$scores.score"},"average":{"$avg":"$scores.score"}}},{"$sort":{"_id":1}}]):
    data.append(j)
print(data)
average.insert_many(data)

[{'_id': 0, 'name': 'aimee Zank', 'total': 49.11994783171814, 'average': 16.373315943906046}, {'_id': 1, 'name': 'Aurelia Menendez', 'total': 184.61969202100374, 'average': 61.53989734033458}, {'_id': 2, 'name': 'Corliss Zuk', 'total': 139.61606947126907, 'average': 46.53868982375636}, {'_id': 3, 'name': 'Bao Ziglar', 'total': 138.70712252233457, 'average': 46.23570750744486}, {'_id': 4, 'name': 'Zachary Langlais', 'total': 203.3963625302728, 'average': 67.79878751009093}, {'_id': 5, 'name': 'Wilburn Spiess', 'total': 134.0186975547794, 'average': 44.67289918492647}, {'_id': 6, 'name': 'Jenette Flanders', 'total': 147.2203575476617, 'average': 49.07345251588723}, {'_id': 7, 'name': 'Salena Olmos', 'total': 229.3959334774732, 'average': 76.46531115915774}, {'_id': 8, 'name': 'Daphne Zheng', 'total': 112.71677331754347, 'average': 37.57225777251449}, {'_id': 9, 'name': 'Sanda Ryba', 'total': 220.08328118626565, 'average': 73.36109372875522}, {'_id': 10, 'name': 'Denisha Cast', 'total': 1

## 5)      Create a new collection which consists of students who scored below average and above 40% in all the categories.

In [38]:
below_average = db.belowavg

In [39]:
below_avg = []
for n in records.aggregate([{"$match":{"$expr":{"$and":[{"$gt":[{"$min":"$scores.score"},40]},{"$lt":[{"$max":"$scores.score"},70]}]}}}]):
    below_avg.append(n)
print(below_avg)

below_average.insert_many(below_avg)

[{'_id': 65, 'name': 'Gena Riccio', 'scores': [{'score': 67.58395308948619, 'type': 'exam'}, {'score': 67.2413500951588, 'type': 'quiz'}, {'score': 42.93471779899529, 'type': 'homework'}]}, {'_id': 85, 'name': 'Rae Kohout', 'scores': [{'score': 63.86894250781692, 'type': 'exam'}, {'score': 55.81549538273672, 'type': 'quiz'}, {'score': 59.13566011309437, 'type': 'homework'}]}, {'_id': 109, 'name': 'Flora Duell', 'scores': [{'score': 40.68238966626067, 'type': 'exam'}, {'score': 46.77972040308903, 'type': 'quiz'}, {'score': 69.29400057020965, 'type': 'homework'}]}, {'_id': 110, 'name': 'Nobuko Linzey', 'scores': [{'score': 67.40792606687442, 'type': 'exam'}, {'score': 58.58331128403415, 'type': 'quiz'}, {'score': 47.44831568815929, 'type': 'homework'}]}, {'_id': 185, 'name': 'Kam Senters', 'scores': [{'score': 49.8822537074033, 'type': 'exam'}, {'score': 45.29515361387067, 'type': 'quiz'}, {'score': 68.88048980292801, 'type': 'homework'}]}]


## 6)      Create a new collection which consists of students who scored below the fail mark in all the categories

In [40]:
fail = db.fail

In [42]:
failure = []
for i in records.aggregate([{"$match":{"$expr":{"$lt":[{"$max":"$scores.score"},40]}}}]):
    failure.append(i)
print(failure)

fail.insert_many(failure)

[{'_id': 0, 'name': 'aimee Zank', 'scores': [{'score': 1.463179736705023, 'type': 'exam'}, {'score': 11.78273309957772, 'type': 'quiz'}, {'score': 35.8740349954354, 'type': 'homework'}]}]


## 7)      Create a new collection which consists of students who scored above pass mark in all the categories.

In [43]:
pass_mark = db.pass_mark

In [44]:
passs = []
for i in records.aggregate([{"$match": {"$expr": {"$gt": [{"$min":"$scores.score"}, 40]}}}]):
    passs.append(i)
print(passs)

pass_mark.insert_many(passs)

[{'_id': 1, 'name': 'Aurelia Menendez', 'scores': [{'score': 60.06045071030959, 'type': 'exam'}, {'score': 52.79790691903873, 'type': 'quiz'}, {'score': 71.76133439165544, 'type': 'homework'}]}, {'_id': 7, 'name': 'Salena Olmos', 'scores': [{'score': 90.37826509157176, 'type': 'exam'}, {'score': 42.48780666956811, 'type': 'quiz'}, {'score': 96.52986171633331, 'type': 'homework'}]}, {'_id': 10, 'name': 'Denisha Cast', 'scores': [{'score': 45.61876862259409, 'type': 'exam'}, {'score': 98.35723209418343, 'type': 'quiz'}, {'score': 55.90835657173456, 'type': 'homework'}]}, {'_id': 11, 'name': 'Marcus Blohm', 'scores': [{'score': 78.42617835651868, 'type': 'exam'}, {'score': 82.58372817930675, 'type': 'quiz'}, {'score': 87.49924733328717, 'type': 'homework'}]}, {'_id': 12, 'name': 'Quincy Danaher', 'scores': [{'score': 54.29841278520669, 'type': 'exam'}, {'score': 85.61270164694737, 'type': 'quiz'}, {'score': 80.40732356118075, 'type': 'homework'}]}, {'_id': 13, 'name': 'Jessika Dagenais', 